<a href="https://colab.research.google.com/github/xuanyu410/114-1PL-Repo/blob/main/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80%E4%BD%9C%E6%A5%AD%E4%BA%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [154]:
# -*- coding: utf-8 -*-
# 確保這些套件已安裝: !pip install google-search-results folium geopy pandas gradio google-genai -q
!pip install google-search-results pandas gradio google-genai -q
import folium
from geopy.geocoders import Nominatim
from serpapi import GoogleSearch
from IPython.display import HTML
import json
import re
import gradio as gr
import pandas as pd
import warnings
from google import genai # 匯入 Gemini SDK
from google.genai.errors import APIError # 匯入錯誤處理

# --- API Key Loading ---
# 請在 Colab Secrets 中設置 'serpapi_key' 和 'int'
try:
    from google.colab import userdata

    # 載入 SerpAPI key (用於地圖搜尋 Tab)
    SERPAPI_KEY = userdata.get('serpapi_key')
    if not SERPAPI_KEY:
        print("🚨 警告：SerpAPI 金鑰（serpapi_key）未從 Colab Secrets 載入。地圖搜尋功能將受限。")

    # 載入 Gemini API key (用於 AI 總結 Tab)
    GEMINI_API_KEY = userdata.get('int') # 使用用戶指定的金鑰名稱 'int'
    if not GEMINI_API_KEY:
        print("🚨 警告：Gemini API 金鑰（int）未從 Colab Secrets 載入。AI 總結功能將使用模擬報告。")
except:
    SERPAPI_KEY = None
    GEMINI_API_KEY = None
    print("🚨 警告：無法從 Colab Secrets 載入 API 金鑰。部分功能將受限。")

In [155]:
# 1. 初始化地理編碼器
geolocator = Nominatim(user_agent="search_map_app")

# 2. 台灣地點列表 (不變)
TAIWAN_LOCATIONS = [
    '台北', '台北市', '臺北', '臺北市', 'Taipei', '新北', '新北市', 'New Taipei',
    '桃園', '桃園市', 'Taoyuan', '台中', '台中市', '臺中', '臺中市', 'Taichung',
    '台南', '台南市', '臺南', '臺南市', 'Tainan', '高雄', '高雄市', 'Kaohsiung',
    '基隆', '基隆市', 'Keelung', '新竹', '新竹市', '新竹縣', 'Hsinchu',
    '苗栗', '苗栗縣', 'Miaoli', '彰化', '彰化縣', 'Changhua', '南投', '南投縣', 'Nantou',
    '雲林', '雲林縣', 'Yunlin', '嘉義', '嘉義市', '嘉義縣', 'Chiayi', '屏東', '屏東縣',
    'Pingtung', '宜蘭', '宜蘭縣', 'Yilan', '花蓮', '花蓮縣', 'Hualien', '台東', '台東縣',
    '臺東', '臺東縣', 'Taitung', '澎湖', '澎湖縣', 'Penghu', '金門', '金門縣', 'Kinmen',
    '連江', '連江縣', '馬祖', 'Matsu',
    '101', '台北101', '西門町', '信義區', '中正區', '士林', '北投', '淡水',
    '九份', '平溪', '日月潭', '阿里山', '墾丁', '太魯閣', '東大門', '逢甲', '一中街', '勤美',
]

# 3. 從 Google Sheet 載入餐廳清單 (df_map_data)
# Google Sheet URL: https://docs.google.com/spreadsheets/d/1UIfts0iHJzLn6VdOeuT3WS7UKDEdS5dylr9WxK1BhFA/edit?gid=1671244919#gid=1671244919
GOOGLE_SHEET_URL = 'https://docs.google.com/spreadsheets/d/1UIfts0iHJzLn6VdOeuT3WS7UKDEdS5dylr9WxK1BhFA/gviz/tq?tqx=out:csv&gid=1671244919'
STANDARD_COLS = ['餐廳名稱', '價位', '距離', '緯度', '經度', '是否營業']

print("⏳ 嘗試從 Google Sheet 載入餐廳清單...")

try:
    df_map_data = pd.read_csv(GOOGLE_SHEET_URL)
    df_map_data.columns = df_map_data.columns.str.strip()

    if not all(col in df_map_data.columns for col in STANDARD_COLS):
        missing_cols = [col for col in STANDARD_COLS if col not in df_map_data.columns]
        print(f"🚨 錯誤：Google Sheet 缺少必要欄位。遺失欄位: {missing_cols}")
        df_map_data = pd.DataFrame(columns=STANDARD_COLS + ['座標'])
    else:
        df_map_data['緯度'] = pd.to_numeric(df_map_data['緯度'], errors='coerce')
        df_map_data['經度'] = pd.to_numeric(df_map_data['經度'], errors='coerce')
        original_len = len(df_map_data)
        df_map_data.dropna(subset=['緯度', '經度'], inplace=True)
        if len(df_map_data) < original_len:
             warnings.warn(f"⚠️ 移除了 {original_len - len(df_map_data)} 筆因缺少經緯度而無法使用的數據。")

        # 使用 '是否營業' 欄位名稱進行布林轉換
        df_map_data['是否營業'] = df_map_data['是否營業'].astype(str).str.lower().str.contains('true|是|1')
        df_map_data['座標'] = list(zip(df_map_data['緯度'], df_map_data['經度']))

        print(f"✅ 餐廳清單 (df_map_data) 已成功載入 {len(df_map_data)} 筆數據。")
        print("載入數據範例:")
        print(df_map_data[['餐廳名稱', '價位', '距離', '座標', '是否營業']].head())

except Exception as e:
    if "401" in str(e):
        print("❌ 載入 Google Sheet 失敗: 權限錯誤 (401)。請確認 Google Sheet 已設為『知道連結的任何人可檢視』。")
    else:
        print(f"❌ 載入 Google Sheet 失敗，發生其他錯誤: {e}")
    df_map_data = pd.DataFrame(columns=STANDARD_COLS + ['座標'])

print("-" * 50)

⏳ 嘗試從 Google Sheet 載入餐廳清單...
✅ 餐廳清單 (df_map_data) 已成功載入 14 筆數據。
載入數據範例:
      餐廳名稱  價位 距離                          座標   是否營業
0    雙月食品社   $  短    (25.0387369, 121.517161)   True
1  溫州街蘿蔔絲餅   $  短         (25.026, 121.53031)   True
2   杭州小籠湯包  $$  中  (25.03650698, 121.5231536)   True
3   瑞安豆漿大王   $  短   (25.0286866, 121.5413439)  False
4     不二煮藝  $$  中  (25.03706557, 121.5499914)   True
--------------------------------------------------


In [156]:
# 4. 初始化 Gemini 客戶端 (使用 GEMINI_API_KEY)
GEMINI_CLIENT = None
if 'GEMINI_API_KEY' in globals() and GEMINI_API_KEY:
    try:
        GEMINI_CLIENT = genai.Client(api_key=GEMINI_API_KEY)
        print("✅ Gemini API 客戶端初始化成功。")
    except Exception as e:
        print(f"🚨 錯誤：Gemini 客戶端初始化失敗: {e}")
        GEMINI_CLIENT = None

# --- 5. SerpAPI 核心函式 ---
def search_google(query, api_key, num=5, gl='tw', hl='zh-tw'):
    try:
        params = {
            'q': query, 'api_key': api_key, 'num': num, 'gl': gl, 'hl': hl
        }
        search = GoogleSearch(params)
        results = search.get_dict()
        return results.get('organic_results', [])
    except Exception as e:
        return []

def extract_locations_from_text(text):
    # ... (函式內容不變) ...
    found_locations = []
    text_lower = text.lower()
    for location in TAIWAN_LOCATIONS:
        if location.lower() in text_lower:
            base_name = location.replace('市', '').replace('縣', '')
            if base_name not in [loc.replace('市', '').replace('縣', '') for loc in found_locations]:
                found_locations.append(location)
    return found_locations

def get_coordinates(location_name):
    # ... (函式內容不變) ...
    try:
        location = geolocator.geocode(f"{location_name}, Taiwan", timeout=10)
        if location:
            return (location.latitude, location.longitude)
        return None
    except Exception as e:
        return None

def analyze_results_with_locations(results):
    # ... (函式內容不變) ...
    analyzed_results = []
    for result in results:
        title = result.get('title', '')
        snippet = result.get('snippet', '')
        combined_text = f"{title} {snippet}"
        locations = extract_locations_from_text(combined_text)
        if locations:
            analyzed_results.append({
                'title': title,
                'snippet': snippet,
                'link': result.get('link', ''),
                'locations': locations
            })
    return analyzed_results

def create_map_from_results(analyzed_results, center=[23.5, 121.0], zoom=7):
    # ... (函式內容不變) ...
    m = folium.Map(location=center, zoom_start=zoom, tiles='OpenStreetMap')
    marked_locations = {}
    for result in analyzed_results:
        for location in result['locations']:
            coords = get_coordinates(location)
            if coords:
                if location not in marked_locations:
                    marked_locations[location] = []
                marked_locations[location].append(result)

    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue']

    for i, (location, results) in enumerate(marked_locations.items()):
        coords = get_coordinates(location)
        if coords:
            lat, lon = coords
            popup_html = f"<div style='width: 300px;'>"
            popup_html += f"<h4 style='color: #2c3e50;'>{location}</h4>"
            popup_html += f"<p style='color: #7f8c8d; font-size: 12px;'>找到 {len(results)} 筆相關結果</p>"
            for j, res in enumerate(results, 1):
                popup_html += f"<div style='margin: 10px 0; padding: 10px; background: #ecf0f1; border-radius: 5px;'>"
                popup_html += f"<b style='color: #2980b9;'>[{j}] {res['title'][:50]}...</b><br>"
                popup_html += f"<small style='color: #34495e;'>{res['snippet'][:100]}...</small><br>"
                popup_html += f"<a href='{res['link']}' target='_blank' style='color: #3498db;'>查看連結</a>"
                popup_html += "</div>"
            popup_html += "</div>"

            folium.Marker(
                location=[lat, lon],
                popup=folium.Popup(popup_html, max_width=350),
                tooltip=f"{location} ({len(results)} 筆結果)",
                icon=folium.Icon(color=colors[i % len(colors)], icon='info-sign')
            ).add_to(m)

    return m

def display_results_summary(analyzed_results):
    # ... (函式內容不變) ...
    summary = ["="*80]
    summary.append(f"找到 {len(analyzed_results)} 筆包含地理位置的搜尋結果")
    summary.append("="*80 + "\n")

    for i, result in enumerate(analyzed_results, 1):
        summary.append(f"【結果 {i}】")
        summary.append(f"標題: {result['title']}")
        summary.append(f"地點: {', '.join(result['locations'])}")
        summary.append(f"描述: {result['snippet'][:100]}...")
        summary.append(f"網址: {result['link']}")
        summary.append("-"*80 + "\n")

    return '\n'.join(summary)

✅ Gemini API 客戶端初始化成功。


In [157]:
# --- 6. Gemini API 呼叫函式 ---

def generate_gemini_summary(recommended_df):
    """使用 Gemini API 根據推薦餐廳生成 AI 總結報告。"""
    if not GEMINI_CLIENT:
        return None

    # 格式化數據作為輸入提示
    restaurant_list = recommended_df[['餐廳名稱', '價位', '距離', '是否營業']].to_markdown(index=False)

    prompt = f"""
    您是一位專業的美食決策顧問。請根據以下推薦的餐廳清單，撰寫一份簡潔、有吸引力的 AI 決策報告（約 150 字，以中文為主）。

    報告內容需包含：
    1. 總結推薦的餐廳數量。
    2. 根據餐廳的『價位』和『距離』欄位，給出決策建議。
    3. 提醒用戶：清單中的餐廳已通過篩選，但仍建議點擊列表查看地圖位置。
    4. 語氣專業、建議具體。

    推薦餐廳清單 (已通過用戶篩選)：
    {restaurant_list}
    """

    try:
        response = GEMINI_CLIENT.models.generate_content(
            model='gemini-2.5-flash',
            contents=prompt,
            config={"temperature": 0.5}
        )
        return "🤖 AI 決策顧問報告 (Gemini):\n" + response.text

    except APIError as e:
        # print(f"🚨 Gemini API 呼叫失敗: {e}") # 避免在 Gradio 介面輸出過多 log
        return None
    except Exception as e:
        # print(f"🚨 呼叫 Gemini 發生未知錯誤: {e}")
        return None

In [158]:
# --- 7. Gradio 互動函式 (wrapper_recommend_with_json 包含 Gemini 邏輯) ---

def generate_folium_map(selected_row_data, df_data, center=[25.03, 121.53], zoom=15):
    """根據 DataFrame 數據生成 Folium 地圖，並標註所有餐廳。"""
    m = folium.Map(location=center, zoom_start=zoom, tiles='OpenStreetMap')

    for index, row in df_data.iterrows():
        lat, lon = row['座標']
        color = 'blue'
        icon = 'cutlery'
        is_selected = (selected_row_data is not None) and (row['餐廳名稱'] == selected_row_data.get('餐廳名稱'))
        if is_selected:
            color = 'red'
            icon = 'star'

        popup_html = f"<b>{row['餐廳名稱']}</b><br>"
        popup_html += f"價位: {row['價位']}<br>"
        popup_html += f"距離: {row['距離']}<br>"
        # 使用 '是否營業' 欄位
        popup_html += f"營業中: {'是' if row['是否營業'] else '否'}"

        folium.Marker(
            location=[lat, lon],
            popup=folium.Popup(popup_html, max_width=250),
            tooltip=row['餐廳名稱'],
            icon=folium.Icon(color=color, icon='cutlery', prefix='fa')
        ).add_to(m)

    return m._repr_html_()

def handle_row_selection(evt: gr.SelectData, recommended_df_json, full_df):
    """處理 DataFrame 行選擇事件，重新繪製地圖並突出顯示被選中的餐廳。"""
    if isinstance(full_df, gr.State):
        full_df = full_df.value

    selected_index = evt.index[0]
    recommended_df = pd.read_json(recommended_df_json)
    selected_name = recommended_df.iloc[selected_index]['餐廳名稱']
    selected_row_data = full_df[full_df['餐廳名稱'] == selected_name].iloc[0].to_dict()

    return generate_folium_map(selected_row_data, full_df)


def wrapper_recommend_with_json(price_filters, distance_filters, available_toggle):
    """根據篩選條件隨機推薦 3 家餐廳，並返回 Gradio 元件所需的所有輸出，AI 總結使用 Gemini API。"""

    if df_map_data.empty:
        empty_df = pd.DataFrame(columns=['餐廳名稱', '價位', '距離'])
        return empty_df, "錯誤：餐廳清單是空的。", "AI 報告: Google Sheet 載入失敗或清單中沒有可用數據。", empty_df.to_json()

    filtered_df = df_map_data.copy()

    # 1. 價格篩選
    if price_filters:
        filtered_df = filtered_df[filtered_df['價位'].isin(price_filters)]

    # 2. 距離篩選
    if distance_filters:
        filtered_df = filtered_df[filtered_df['距離'].isin(distance_filters)]

    # 3. 營業中篩選
    if available_toggle:
        filtered_df = filtered_df[filtered_df['是否營業'] == True]

    if filtered_df.empty:
        empty_df = pd.DataFrame(columns=['餐廳名稱', '價位', '距離'])
        return empty_df, "找不到符合條件的餐廳。", "AI 報告: 沒有符合所有篩選條件的餐廳，請放寬條件。", empty_df.to_json()

    # 4. 隨機推薦 3 家
    num_recommend = min(3, len(filtered_df))
    try:
        recommended_df = filtered_df.sample(n=num_recommend, random_state=42)
    except ValueError as e:
        recommended_df = filtered_df.head(num_recommend)

    # 5. 準備 Gradio 輸出數據
    output_df = recommended_df[['餐廳名稱', '價位', '距離']]
    output_list = "\n".join([
        f"{i+1}. {row['餐廳名稱']} (價位: {row['價位']}, 距離: {row['距離']}, 營業中: {'是' if row['是否營業'] else '否'})"
        for i, row in recommended_df.iterrows()
    ])


    # === 【使用 Gemini API 進行 AI 報告總結】 ===
    ai_summary = None # 確保這裡只有一層縮排

    if GEMINI_CLIENT:
        gemini_result = generate_gemini_summary(recommended_df)
        if gemini_result:
            ai_summary = gemini_result

    if ai_summary is None:
        # 如果 GEMINI_CLIENT 未配置、呼叫失敗，則使用模擬報告
        ai_summary = f"🍽️ AI 決策顧問報告 (模擬報告或 API 呼叫失敗)：\n本次推薦 {num_recommend} 家餐廳。\n\n優點：推薦餐廳皆符合您設定的價格和距離要求。\n建議：點擊列表中的餐廳，查看地圖位置後再決定。"

    # === 【Gemini 總結結束】 ===

    recommended_df_json = recommended_df.to_json()

    return output_df, output_list, ai_summary, recommended_df_json

In [159]:
# --- SerpAPI 驅動主控函式 (不變) ---
def serpapi_search_and_map_wrapper(query, num_results=10):
    # ... (函式內容不變) ...
    if not SERPAPI_KEY:
        error_msg = "錯誤：SerpAPI 金鑰（SERPAPI_KEY）未定義。請檢查您的 Colab Secrets 設置。"
        return None, error_msg

    try:
        results = search_google(query, SERPAPI_KEY, num=num_results)

        if not results:
            return None, "沒有找到任何搜尋結果。"

        analyzed_results = analyze_results_with_locations(results)

        if not analyzed_results:
            return None, "\n⚠️ 搜尋結果中沒有找到可辨識的台灣地區"

        summary = display_results_summary(analyzed_results)
        map_obj = create_map_from_results(analyzed_results)
        map_html = map_obj._repr_html_()

        return map_html, summary

    except Exception as e:
        return None, f"發生錯誤: {e}"

In [160]:
# 設定 Gradio 介面元件
with gr.Blocks(title="師大午餐決策器") as demo:
    gr.Markdown(
        """
        # 🍽️ 師大午餐/聚會決策器
        ### 目標：從餐廳清單中篩選，並隨機推薦 3 家高cp質餐廳，並提供台灣各地美食資訊搜尋！
        """
    )

    # 1. 餐廳決策 Tab
    with gr.Tab("🍽️ 餐廳決策與 AI 總結"):

        # 狀態元件：用於傳輸 DataFrame 資料給地圖選擇器
        recommended_df_json = gr.State(value=pd.DataFrame().to_json())

        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("## 條件篩選")
                price_select = gr.CheckboxGroup(
                    ['$', '$$', '$$$'],
                    label="價位過濾 (可複選)",
                    value=['$', '$$'],
                    info="($: 銅板/150內, $$: 150-300, $$$: 300+)"
                )
                distance_select = gr.CheckboxGroup(
                    ['短', '中', '長'],
                    label="距離過濾 (可複選)",
                    value=['短', '中'],
                    info="(短: 步行5分, 中: 步行10分, 長: 需騎車)"
                )
                available_toggle = gr.Checkbox(
                    label="只看是否營業", # 配合 '是否營業' 欄位名稱
                    value=True
                )
                recommend_btn = gr.Button("🚀 開始推薦")

            with gr.Column(scale=2):
                gr.Markdown("## 推薦清單 (隨機 3 家)")
                output_dataframe = gr.DataFrame(
                    headers=["餐廳名稱", "價位", "距離"],
                    datatype=["str", "str", "str"],
                    label="隨機推薦結果",
                    type="pandas",
                    interactive=True,
                )
                output_list = gr.Textbox(label="推薦清單細節")

        gr.Markdown("## 📍 餐廳位置地圖 (點擊上方列表中的餐廳)")
        map_output = gr.HTML(
            generate_folium_map(None, df_map_data),
            label="互動式地圖",
            elem_id="map_container"
        )

        gr.Markdown("## 🤖 AI 決策建議 (約 150 字)")
        output_summary = gr.Textbox(
            label="AI 決策顧問報告",
            interactive=False,
            info="AI 將根據推薦結果，總結優缺點，幫助您快速做出選擇。"
        )

        # 綁定事件
        recommend_btn.click(
            fn=wrapper_recommend_with_json,
            inputs=[price_select, distance_select, available_toggle],
            outputs=[output_dataframe, output_list, output_summary, recommended_df_json]
        )

        # 修正後的 Gradio 綁定順序
        output_dataframe.select(
            fn=handle_row_selection,
            inputs=[recommended_df_json, gr.State(df_map_data)], # 正確順序：(JSON, 完整 DataFrame)
            outputs=[map_output]
        )


    # 2. 台灣地圖搜尋 Tab
    with gr.Tab("🗺️ 台灣地圖搜尋"):
        gr.Markdown("## ✨ Google 搜尋結果地圖化 (SerpAPI 驅動)")
        gr.Markdown("輸入關鍵字 (例如：台北必吃拉麵)，程式將自動解析結果中的台灣地點並在地圖上標註。")

        with gr.Row():
            query_input = gr.Textbox(label="輸入搜尋關鍵字", value="台南熱門小吃")
            num_input = gr.Slider(minimum=5, maximum=20, step=1, label="搜尋結果數量", value=10)
            search_map_btn = gr.Button("🔍 搜尋並繪製地圖")

        map_search_output = gr.HTML(
            label="搜尋結果地圖",
            elem_id="search_map_container"
        )

        summary_search_output = gr.Textbox(
            label="搜尋結果摘要",
            interactive=False,
            lines=10
        )

        # 綁定 SerpAPI 搜尋功能
        search_map_btn.click(
            fn=serpapi_search_and_map_wrapper,
            inputs=[query_input, num_input],
            outputs=[map_search_output, summary_search_output]
        )

# 運行 Gradio 應用
if __name__ == "__main__":
    print("Gradio 應用正在啟動...")
    demo.launch(inbrowser=True, share=True)

Gradio 應用正在啟動...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://704b50681a91da7c3d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
